# Yandex.Cloud

Панель управления облаком через браузер - https://cloud.yandex.ru

Каждому студенту выделена отдельная директория в общем облаке. Внутри этой директории можно создавать различные облачные ресурсы. Все операции для работы с облаком можно проводить из браузера или из консоли.

Чтобы работать из консоли, необходимо установить утилиту **yc**.

Полный гайд есть на сайте - https://cloud.yandex.ru/docs/cli/quickstart

#### Но перед этим, если у вас Windows...

1. Мы вам искренне соболезнуем
2. Установите на компьютер WSL (Windows Subsystem Linux) - https://docs.microsoft.com/ru-ru/windows/wsl/install . Если это у вас успешно получится, то из этой среды вы сможете делать все те действия, что описываются далее

Краткая выжимка того, что нужно сделать, чтобы все начало работать.

1) Устанавливаем клиент 

```bash
curl https://storage.yandexcloud.net/yandexcloud-yc/install.sh | bash
```

2) Авторизуемся 

```bash
yc init
```

В процессе авторизации вам нужно будет открыть браузер, авторизоваться в облаке с помощью вашей почты на yandex.ru (которую вы указывали в форме), перекопировать токен и выбрать вашу директорию для lsml.

3) Проверяем, что все сработало 

```bash
yc config list
```

4) Вы великолепны!

Для дальнейшей работы с облаком вам может потребоваться ssh ключ. Если он у вас есть (посмотрите в `~/.ssh`) , то все отлично! Если нет, то сгенерируйте ключ (хорошая инструкция например от github - https://docs.github.com/en/authentication/connecting-to-github-with-ssh/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent )

# Подготавливаем облако к продуктивной работе

## Сервисный аккаунт

В вашем каталоге уже должен быть настроенный сервисный аккаунт `lsml2022account-x`. Используйте его во всех ресурсах, где требуется его указать.

## Настраиваем сеть

Без настроенной сети работать с облаком будет затруднительно. Поэтому давайте настроем сеть в облаке.

Идем в `Virtual Private Cloud`, жмем создать сеть, называем ее `default` и кликаем `Создать подсети`.

Потом идем в подсети и для `default-ru-central1-a` включаем NAT (это нужно, чтобы мы потом могли подключиться с ноутбука к кластеру).


## Настраиваем хранилище

Вся наша бигдата должна где-то хранится. Для этого создадим бакет в `Object Storage`. Название бакету можете выдать любое. Единственное ограничение - оно должно быть унивальным во всем Яндекс.Облаке.

Выставляйте ему следующие настройки

* Лимит - 1 ТБ
* Доступ на чтение объектов - публичный (это нужно, чтобы мы потом могли проверить ваши домашки)
* Доступ к списку объектов - ограниченный
* Доступ на чтение настроек - ограниченный
* Класс хранилища - стандартное


## Настраиваем кластер 

Время настраивать сам кластер. Идем в `Data Proc`. Кстати, в ```yandex-cloud-terraform``` лежит конфиг терраформа, который поможет поднять все гораздо легче.

* Выключаем `TEZ`
* Включаем `Livy`
* Зона доступности - `central1-a`
* Имя бакета - тот, что создали до этого
* Сеть - ту, что создали до этого
* ssh-ключ - ваш ключ с суффиксом .pub
* `UI Proxy` - включаем
* Ниже можно добавить подкластер с `ComputeNode` - там можно указать, сколько машин для рассчетов добавить в кластер (для начала можно и одну, но в будущем скорее всего потребуется добавлять больше, чтобы считать задачи)
    * Выставляйте hdd диски вместо ssd для машин, а то может не хватить на всех
    * Сильно большие тачки лучше не создавать - парочка `s2.small` это вполне неплохой выбор для наших целей

**Note:** ComputeNode - это подкластер, где можно запускать Spark поверх данных из объектоного хранилища, DataNode - это подкластер, где можно к дополнение к этому еще запускать и классический Хадуп поверх локальный данных на машине (в HDFS на машинах). На данном этапе разницы никакой - как лучше настраивать кластер мы поймем дальше по ходу курса. 

**Ждем...**

Ребята в Яндексе постарались и поэтому кластер создается достаточно быстро, но все таки это занимает какое-то время. Более того, наливка кластера - задача не самая простая, поэтому есть шанс увидеть кластер в состоянии `Dead` - пугаться не стоит, скоро оно починится и вы с можете начать с ним работать.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/1/umer.png" width="200px">

Когда чудо произойдет, можно провести простой тест и попробовать открыть какие-то UI Proxy.

## Настраиваем Proxy машину

Работы напрямую с Hadoop необходимо проворачивать с MasterNode. Из-за соображений безопасности, нельзя просто так подключится к мастер-ноде. Для того, чтобы сделать это, нам потребуется прокси.

Идем в `Compute Cloud`

Создаем машину минимального размера (она потребуется только для проксирования)

* Зона доступности - central1-a
* Ubuntu
* Подсеть - central1-a
* Сервисный аккаунт - тот самый
* Логин - yc-user
* ssh-ключ - тот же самый ваш публичный
* Гарантированная доля vCPU - 20%. Это снижает расходы на тачку в ~1.5 раза

После создания копируем публичный IP адрес машины и пробуем подключиться (ниже мои IP для примера - у себя используйте свои)

```bash
ssh yc-user@62.84.126.222
```

Если все хорошо, то пытаемся подключиться через прокси-машину до мастера нашего кластера. Сначала добавляем наш ключик в ssh-агент следующей командой:

```bash
ssh-add ~/.ssh/id_key
```

Дальше нам надо найти внутренний FQDN мастер ноды (его можно посмотреть, кликнув на машину мастер ноды).

А теперь вишенка на торте:

```bash
ssh -A -J yc-user@62.84.126.222 ubuntu@rc1a-dataproc-m-81gv9d0a18t0dy1r.mdb.yandexcloud.net
```

Эта команда автоматически подключит нас к мастеру через прокси-сервер. Магия, да и только! Теперь можно упростить подключение, добавив в ~/.ssh/config конфигурацию нашего подключения.

```
echo <<EOF >>~/.ssh/config
Host lsml-proxy
	HostName 62.84.126.222
	User yc-user
	IdentityFile ~/.ssh/id_key

Host lsml-head
    HostName rc1a-dataproc-m-81gv9d0a18t0dy1r.mdb.yandexcloud.net
	User ubuntu
	ProxyJump lsml-proxy
EOF
```

После этого подключение к головному узлу становится максимально простым:

```bash
ssh lsml-head
```

Проверяем, что все завелось
```bash
hadoop version

hdfs dfs -ls /
```

Если все хорошо, давайте сразу проверим работу с `Object Storage`. 

Пробуем запустить комманду (`lsml2022data` - название моего бакета)

```bash
hdfs dfs -ls s3a://lsml2022data/
```

Это мы попробовали прочитать что-то из бакета. Если все прошло успешно, давайте попробуем записать что-то.

```bash
echo "Biba" > kuka.txt
hdfs dfs -put kuka.txt s3a://lsml2022data/kuka.txt
```

Если все успешно прошло, то перейдите в админке облака в `Object Storage` и посмотрите в него - там должен оказаться файл kuka.txt с содержанием "Biba"

## Настраиваем проект в Data Sphere

Для удобного управление вычислениями в кластере, создадим проект в ДатаСфере и будем работать оттуда.

При создании, заполните также "Дополнительные параметры":
* Сервисный аккаунт - тот самый
* Подсеть - default-ru-central1-a
* Кластер `Data Proc` - тот, что создали до этого

После создания Jupyter Lab создайте там новый ноутбук (и сразу дайте ему какое-то осмысленное название!) и запустите что-нибудь вроде

```python
print("Hello pupa, I am the lupa")
```

После этого должна начать создаваться машина для работы ноутбука. Как только все свершится, команда должна запуститься.

По умолчанию в датасфере все вычисления происходят на выделенной машине с собственным интерпретатором без кластеров и всего такого.

Для того, чтобы начать запускать команды в контексте спарка, необходимо добавить специальный макрос в начале клетки.

In [1]:
#!spark

print("Run on Livy")

Run on Livy


In [ ]:
#!spark

spark

In [ ]:
#!spark

sc = spark.sparkContext

In [ ]:
#!spark

data = sc.textFile("s3a://lsml2022data/kuka.txt")

In [ ]:
#!spark

data.collect()

После запуска этих ячеек спарк должен был прочитать данные из созданного нами ранее файла.

## Действия после работы

Работающий кластер кушает много денег. Поэтому чтобы они не закончились, останавливайте кластер после использования. Остальные запчасти в облаке можно не трогать, они кушают мало денег.

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2022-public/main/data/1/5a3867946daef3.6346339815136459724493.png">


# Аплодисменты 

Поздравляю, теперь вы готовы к сражению с биг датой! **И не забудьте погасить свой кластер.**